In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import isnan, when, count, col

In [ ]:
spark = SparkSession.builder.appName('CaliHousing').getOrCreate()

# **Part 1 - Data Preprocessing**

In [ ]:
df = spark.read.csv('/FileStore/tables/1990californiahousing.csv', inferSchema = True, header = True)

df.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
only showing top 5 rows

In [ ]:
df.select('median_house_value').describe().show() #20640 rows in total with mean value of $206855 for housing price

+-------+------------------+
summary|median_house_value|
+-------+------------------+
 count| 20640|
 mean|206855.81690891474|
 stddev|115395.61587441359|
 min| 14999.0|
 max| 500001.0|
+-------+------------------+

**1.1 Cleaning Null Values**

In [ ]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show() #looks like 207 null values from total_bedrooms

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
 0| 0| 0| 0| 207| 0| 0| 0| 0| 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+

In [ ]:
df = df.na.drop(how = 'any') #so dropping rows where all the row values are null
df.select('median_house_value').describe().show() #see we dropped 207 null rows 

+-------+------------------+
summary|median_house_value|
+-------+------------------+
 count| 20433|
 mean|206864.41315519012|
 stddev|115435.66709858322|
 min| 14999.0|
 max| 500001.0|
+-------+------------------+

**1.2 Encoding Categorial Data**

In [ ]:
df.groupBy('ocean_proximity').count().show() #so we got 5 categories for ocean proximity

+---------------+-----+
ocean_proximity|count|
+---------------+-----+
 ISLAND| 5|
 NEAR OCEAN| 2628|
 NEAR BAY| 2270|
 <1H OCEAN| 9034|
 INLAND| 6496|
+---------------+-----+

In [ ]:
encoder = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_cat")
encoded = encoder.fit(df).transform(df)
encoded.show(5) #notice new column that encodes the ocean prox data

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_cat|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
 -122.23| 37.88| 41.0| 880.0| 129.0| 322.0| 126.0| 8.3252| 452600.0| NEAR BAY| 3.0|
 -122.22| 37.86| 21.0| 7099.0| 1106.0| 2401.0| 1138.0| 8.3014| 358500.0| NEAR BAY| 3.0|
 -122.24| 37.85| 52.0| 1467.0| 190.0| 496.0| 177.0| 7.2574| 352100.0| NEAR BAY| 3.0|
 -122.25| 37.85| 52.0| 1274.0| 235.0| 558.0| 219.0| 5.6431| 341300.0| NEAR BAY| 3.0|
 -122.25| 37.85| 52.0| 1627.0| 280.0| 565.0| 259.0| 3.8462| 342200.0| NEAR BAY| 3.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+
only showing top 5 rows

**1.3 Creating Features Column**

In [ ]:
assembler = VectorAssembler(inputCols=['longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income',
                                       'ocean_proximity_cat'],
                           outputCol='features') 
#so we're grabbing our relevant features and turning them into a single vector, since PySpark need 2 columns coming in as Features and Label. OutputCol is the single feature column that we combined

output = assembler.transform(encoded) #applying our vector assembler to all our data

output.printSchema()

root
-- longitude: double (nullable = true)
-- latitude: double (nullable = true)
-- housing_median_age: double (nullable = true)
-- total_rooms: double (nullable = true)
-- total_bedrooms: double (nullable = true)
-- population: double (nullable = true)
-- households: double (nullable = true)
-- median_income: double (nullable = true)
-- median_house_value: double (nullable = true)
-- ocean_proximity: string (nullable = true)
-- ocean_proximity_cat: double (nullable = false)
-- features: vector (nullable = true)

In [ ]:
output.select('features').head(1) #notice features is a DenseVector containing all the features we combined

Out[10]: [Row(features=DenseVector([-122.23, 37.88, 41.0, 880.0, 129.0, 322.0, 126.0, 8.3252, 3.0]))]

**1.4 Creating Feature and Label Dataframe**


In [ ]:
final_data = output.select('features', 'median_house_value') #so the data we'll be using to apply LR will be the features and the label

final_data.show(3)

+--------------------+------------------+
 features|median_house_value|
+--------------------+------------------+
[-122.23,37.88,41...| 452600.0|
[-122.22,37.86,21...| 358500.0|
[-122.24,37.85,52...| 352100.0|
+--------------------+------------------+
only showing top 3 rows

**1.5 Train Test Split**


In [ ]:
train, test = final_data.randomSplit([0.8,0.2]) #train will have 80%, test will have 20% of data

train.describe().show()

+-------+------------------+
summary|median_house_value|
+-------+------------------+
 count| 16351|
 mean|206961.38187266834|
 stddev|115265.10632236877|
 min| 14999.0|
 max| 500001.0|
+-------+------------------+

In [ ]:
test.describe().show()

+-------+------------------+
summary|median_house_value|
+-------+------------------+
 count| 4082|
 mean| 206475.9919157276|
 stddev|116129.73170017391|
 min| 22500.0|
 max| 500001.0|
+-------+------------------+

# **Part 2 - Creating Model**

In [ ]:
lr = LinearRegression(featuresCol='features', labelCol='median_house_value', predictionCol='prediction', maxIter=200) #Creating instance of linear regression module

lrmodel = lr.fit(train) #fitting our train data

# **Part 3 - Evaluating Model**

In [ ]:
test_results = lrmodel.evaluate(test)

test_results.residuals.show() #residuals is the diff between predicted values and label

+-------------------+
 residuals|
+-------------------+
 -44861.2343934346|
 11069.452541896142|
 -62174.13952363422|
 -71873.2328862315|
 -48922.51119645266|
 126480.80096466746|
 -51880.57862927765|
 49805.388888827525|
 -65902.26206663577|
 -81494.49987139832|
-47744.521984169725|
 -2813.632734784391|
-7309.4298051008955|
-17334.447228764184|
 57124.157108515035|
 -78966.59478046|
 -40923.31040235562|
 -34775.53556794254|
 2327.7337433062494|
 25305.732973218895|
+-------------------+
only showing top 20 rows

In [ ]:
test_results.rootMeanSquaredError #we're off on average by $71944.62 for housing price, so we're within one standard deviation of $116129.73

Out[35]: 71944.61620404138

In [ ]:
final_data.describe().show() #we're within one standard deviation

+-------+------------------+
summary|median_house_value|
+-------+------------------+
 count| 20433|
 mean|206864.41315519012|
 stddev|115435.66709858322|
 min| 14999.0|
 max| 500001.0|
+-------+------------------+